# Packages

In [1]:
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
import requests
import json

import country_converter as coco
cc = coco.CountryConverter()

# Composite Leading Indicators (MEI): OCDE

## Data

In [2]:
df = web.DataReader(
    'MEI_CLI', # https://stats.oecd.org/Index.aspx?DataSetCode=MEI_CLI
    'oecd', 
    start='2007-01-01'
)
df = df.transpose().loc[(
    "Amplitude adjusted (CLI)",
    #"Normalised (CLI)",
    #"Normalised (GDP)",
    #"Trend restored (CLI)",
),:]
df = df.reset_index()
df = df[df["Frequency"] == "Monthly"].drop(["Frequency"], axis='columns')
df = df.transpose()
df.columns = cc.convert(names = list(df.loc["Country"]), to = 'ISO3', not_found=None) # To avoid string problems
df.columns.name = ''
df.drop(['Country'], inplace=True)
df.index.name = ''
df.index = pd.date_range(
    start = df.index[0],
    end = f"{str(df.index[-1])[:6]}{int(str(df.index[-1])[6])+1}{str(df.index[-1])[7:]}",
    freq='M', 
)
df.to_csv('../raw/CLI.csv')
df.tail()

/home/gpetrini/.local/lib/python3.8/site-packages/pandas/core/indexing.py:873: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


ValueError: could not convert string to Timestamp

## Visualization

In [ ]:
cases = ["USA", "ESP", "ITA", "G7", "OECD total "]
source = f"Source: OECD\nLast query: {dt.today():%d/%m/%y}"

fig, ax = plt.subplots(figsize=(8,5))

df['2018':][cases].plot(ax=ax, lw=2)
df['2018':][["BRA"]].plot(ax=ax, lw=3, color='darkred',)
ax.set_title("Composite Leading Indicators (MEI)\nAmplitude adjusted", fontweight='bold')
fig.text(0.79, .28, source, ha='left')
ax.axvline(x='2020-03-18', label='More than 60\ncases in Brazil',
           ls='--', color='black', lw=1.5, )
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
sns.despine()
plt.show()

# Consumer confidence index (CCI)

In [ ]:
df = web.DataReader(
    'MEI_CLI', # https://stats.oecd.org/Index.aspx?DataSetCode=MEI_CCI
    'oecd', 
    start='2007-01-01'
)
df = df.transpose().loc[(
    'OECD Standardised CCI, Amplitude adjusted (Long term average=100), sa',
    #"Normalised (CLI)",
    #"Normalised (GDP)",
    #"Trend restored (CLI)",
),:]
df = df.reset_index()
df = df[df["Frequency"] == "Monthly"].drop(["Frequency"], axis='columns')
df = df.transpose()
df.columns = cc.convert(names = list(df.loc["Country"]), to = 'ISO3', not_found=None) # To avoid string problems
df.columns.name = ''
df.drop(['Country'], inplace=True)
df.index.name = ''
df.index = pd.date_range(
    start = df.index[0],
    end = f"{str(df.index[-1])[:6]}{int(str(df.index[-1])[6])+1}{str(df.index[-1])[7:]}",
    freq='M', 
)
df.to_csv('../raw/CCI.csv')
df.tail()

## Visualization

In [ ]:
cases = ["USA", "ESP", "ITA", "G7", "OECD total "]
source = f"Source: OECD\nLast query: {dt.today():%d/%m/%y}"

fig, ax = plt.subplots(figsize=(8,5))

df['2019':][cases].plot(ax=ax, lw=2)
df['2019':][["BRA"]].plot(ax=ax, lw=3, color='darkred',)
ax.set_title("Índice de Confiança do Consumidor (CCI)", fontweight='bold')
fig.text(0.79, .28, source, ha='left')
ax.axvline(x='2020-03-24', label='Início do Isolamento em SP',
           ls='--', color='black', lw=1.5, )
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
sns.despine()
plt.show()

# GDP growth rates (QoQ)

## Data

In [ ]:

df = pd.read_csv(
    '../raw/QNAraw.csv',
)
df = df[['Value', "TIME", "LOCATION"]]
df = pd.pivot_table(
    df, 
    values="Value", 
    index="TIME", 
    columns="LOCATION"
)
df.index = pd.date_range(
    start = "31/03/2018",
    end = "31/03/2020",
    freq='Q', 
)
df.index.name=''
df.columns.name=''
df.to_csv('../raw/GDP_QoQ_Growth.csv')
df

## Visualization

In [ ]:
cases = ["USA", "ESP", "ITA", ]
source = f"Source: OECD\nLast query: {dt.today():%d/%m/%y}"

fig, ax = plt.subplots(figsize=(8,5))

df[cases].plot(ax=ax, lw=2)
df[["BRA"]].plot(ax=ax, lw=3, color='darkred',)
ax.set_title("GDP Growth rate QoQ\nSeasonally adjusted", fontweight='bold')
fig.text(0.88, .35, source, ha='left')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
sns.despine()
plt.show()

In [ ]:
# df = web.DataReader(
#     'QNA', # https://stats.oecd.org/Index.aspx?DataSetCode=QNA
#     'oecd', 
#     start=dt(2018,3,31),
#     end=dt(2018,6,30)
# )
# df = df.transpose().loc[(
#     'B1_GE: Gross domestic product - expenditure approach',
#     'GPSA: Growth rate compared to previous quarter, seasonally adjusted',
#     #'GYSA: Growth rate compared to the same of previous year, seasonally adjusted',
#     'Quarterly'
# ),:]
# df = df.reset_index()
# df = df[df["Frequency"] == "Quarterly"].drop(["Frequency"], axis='columns')
# df = df.transpose()
# df